# Load data from Smart-Kages into `movement` datasets
Load all DLC .h5 pose files for each kage and concatenate them
into a single `movement` dataset.

Assign a datetime index across the `time` dimension for easy access.

Save the resulting dataset to a netCDF file.

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
from matplotlib import pyplot as plt
from movement.io import load_poses

from smart_kages_movement.io import load_background_frame

## Configuration
Define some global variables and paths.

In [2]:
FPS = 2  # frames per second
PIXELS_PER_CM = 10  # pixels per centimetre (need to double-check this)
TIME_PRECISION = "ns"

# Configure seaborn for prettier plots
sns.set_context("notebook")
sns.set_style("ticks")

In [ ]:
data_dir = Path.home() / "Data" / "Smart-Kages"
analysis_dir = data_dir / "movement_analysis"
# csv file generated by the 01 notebook
df_path = analysis_dir / "all_segments.csv"


for path in [data_dir, analysis_dir, df_path]:
    if not path.exists():
        print(f"Path does not exist: {path}")

## Load CSV files as dataframe

We load the dataframe containing the paths to all the DLC .h5 files, which is generated in the 01 notebook.

In [4]:
df = pd.read_csv(
    df_path,
    index_col=[0, 1, 2],
    dtype={
        "date": str,
        "hour": str,
        "n_frames": int,
    },
    parse_dates=["start_datetime", "end_datetime"],
)
df.head()

start_datetime                  end_datetime  \
kage   date     hour                                                     
kage10 20240403 00   2024-04-03 00:00:06 2024-04-03 00:59:59.493364484   
                01   2024-04-03 01:00:05 2024-04-03 01:59:59.506578626   
                02   2024-04-03 02:00:06 2024-04-03 02:59:59.494565316   
                03   2024-04-03 03:00:06 2024-04-03 03:59:59.495557178   
                04   2024-04-03 04:00:06 2024-04-03 04:59:59.495144236   

                      n_frames  n_channels  height  width  \
kage   date     hour                                        
kage10 20240403 00        7189           3     376    500   
                01        7191           3     376    500   
                02        7189           3     376    500   
                03        7189           3     376    500   
                04        7189           3     376    500   

                                                         pose_file_path  \
kage   date     hour                                                      
kage10 20240403 00    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                01    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                02    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                03    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                04    /ceph/neuroinformatics/neuroinformatics/sirmpi...   

                                                        video_file_path  \
kage   date     hour                                                      
kage10 20240403 00    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                01    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                02    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                03    /ceph/neuroinformatics/neuroinformatics/sirmpi...   
                04    /ceph/neuroinformatics/neuroinformatics/sirmpi...   

                                                   timestamps_file_path  
kage   date     hour                                                     
kage10 20240403 00    /ceph/neuroinformatics/neuroinformatics/sirmpi...  
                01    /ceph/neuroinformatics/neuroinformatics/sirmpi...  
                02    /ceph/neuroinformatics/neuroinformatics/sirmpi...  
                03    /ceph/neuroinformatics/neuroinformatics/sirmpi...  
                04    /ceph/neuroinformatics/neuroinformatics/sirmpi...

We see that the dataframe holds various information on each 1-hour segment of the data.

The index is hierarchical, organising the data first by `kage`, then by `date`, and finally by `hour`.

Of special relevance to us here:
- Paths: `pose_file_path`, `video_file_path`, `timestamps_file_path`
- `start_datetime` and `end_datetime` for each segment

## Load all data from a given kage
We will create a function that, given a kage name, will load all the data from the DLC .h5 files and merge them into a single `movement` dataset.

In [5]:
def kage_to_movement_ds(
    df: pd.DataFrame,
    kage: str,
) -> tuple[xr.Dataset, np.ndarray]:
    """Load all poses for a given kage and return an xarray Dataset.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the paths to pose files as well as metadata
        for each 1-hour segment.
    kage : str
        The name of the kage to process, e.g., "kage1", "kage2", etc.

    Returns
    -------
    xr.Dataset
        An xarray Dataset containing the poses for the specified kage,
        with time coordinates assigned based on the corrected timestamps.
    np.ndarray
        A background image (numpy array) loaded from the middle segment
        of the kage, used for visualization purposes.

    Notes
    -----
    The returned Dataset will have two time coordinates:
    - ``time``: the primary time coordinate based on datetime timestamps.
    - ``seconds_since_start``: the secondary time coordinate representing
       seconds elapsed since the start of the kage.
    """

    def _is_monotonic_increasing(arr):
        """Check if a 1D array is monotonically increasing."""
        return (arr[1:] >= arr[:-1]).all()

    print(f"Processing kage: {kage}")
    df_kage = df.loc[kage]
    df_kage = df_kage.sort_index()  # ensure chronological order
    n_days = df_kage.index.get_level_values("date").nunique()
    print(f"Number of days: {n_days}")
    n_segments = df_kage.shape[0]
    print(f"Number of 1-hour segments: {n_segments}")

    kage_start_datetime = pd.Timestamp(
        df_kage["start_datetime"].iloc[0], unit=TIME_PRECISION
    )

    ds_segments = []  # List of xarray Datasets for each 1-hour segment

    for date, hour in df_kage.index:
        # Load the pose data for the current 1-hour segment
        poses = load_poses.from_file(
            df_kage.loc[(date, hour), "pose_file_path"],
            source_software="DeepLabCut",
            fps=FPS,
        )

        # Assert that length of tracks matches the number of video frames
        n_frames = df_kage.loc[(date, hour), "n_frames"]
        assert poses.sizes["time"] == n_frames, (
            f"Number of tracked timepoints ({poses.sizes['time']}) does not "
            f"match the number of frames ({n_frames}) for {date} at {hour}!"
        )

        # Load datetime timestamps from file (in ISO format)
        timestamps = pd.read_csv(
            df_kage.loc[(date, hour), "timestamps_file_path"],
            header=None,
            index_col=False,
            parse_dates=[0],
        ).iloc[:, 0].values

        # assign time coordinates to the actual datetime timestamps
        poses = poses.assign_coords(time=timestamps)
        poses.attrs["time_unit"] = f"datetime64[{TIME_PRECISION}]"

        # add to list of loaded segments
        ds_segments.append(poses)

    # Combine all segments into a single xarray Dataset
    ds_kage = xr.concat(ds_segments, dim="time")
    ds_kage.attrs["kage"] = kage
    ds_kage.attrs["kage_start_datetime"] = kage_start_datetime.isoformat()

    # Ensure the concatenated timestamps are monotonic increasing
    assert _is_monotonic_increasing(ds_kage.time.values), (
        f"Combined timestamps for {kage} are not monotonic increasing!"
    )

    # Assign secondary time coordinate as seconds elapsed since kage start
    seconds_since_kage_start = (
        ds_kage.time.data - np.datetime64(kage_start_datetime)
    ) / pd.Timedelta("1s")
    ds_kage = ds_kage.assign_coords(
        seconds_elapsed=("time", seconds_since_kage_start)
    )

    # load image to use as background frame
    video_path = df_kage.iloc[n_segments // 2]["video_file_path"]
    background_img = load_background_frame(
        video_path=video_path, i=0, n_average=100
    )
    print(f"Loaded background image for {kage} from {video_path} \n")

    return ds_kage, background_img

Now let's create a combined `movement` dataset for each kage.

We also assign a background image to each kage, which is used for visualisation purposes.

In [6]:
ds_dict = {}  # List of `movement` datasets for each kage
img_dict = {}  # List of background images for each kage

for kage in df.index.get_level_values("kage").unique():
    ds_dict[kage], img_dict[kage] = kage_to_movement_ds(df, kage)

Processing kage: kage10
Number of days: 38
Number of 1-hour segments: 859
Loaded background image for kage10 from /ceph/neuroinformatics/neuroinformatics/sirmpilatzen/behav_data/Loukia/SmartKages/kage10/videos/2024/04/21/kage10_20240421_010002.mp4 



In [7]:
# Inspect the dataset for kage1
ds_dict["kage10"]

<xarray.Dataset> Size: 1GB
Dimensions:          (time: 6163466, space: 2, keypoints: 8, individuals: 1)
Coordinates:
  * space            (space) <U1 8B 'x' 'y'
  * keypoints        (keypoints) <U10 320B 'snout' 'leftear' ... 'tailbase'
  * individuals      (individuals) <U12 48B 'individual_0'
  * time             (time) datetime64[ns] 49MB 2024-04-03T00:00:06 ... 2024-...
    seconds_elapsed  (time) float64 49MB 0.0 0.4999 ... 3.226e+06 3.226e+06
Data variables:
    position         (time, space, keypoints, individuals) float64 789MB 283....
    confidence       (time, keypoints, individuals) float64 394MB 0.0006478 ....
Attributes:
    source_software:      DeepLabCut
    ds_type:              poses
    fps:                  2.0
    time_unit:            datetime64[ns]
    source_file:          /ceph/neuroinformatics/neuroinformatics/sirmpilatze...
    kage:                 kage10
    kage_start_datetime:  2024-04-03T00:00:06

## Save the combined datasets to netCDF files
Finally, we save the combined `movement` datasets for each kage to a netCDF file.

We also save the background image for each kage in the same directory.

In [8]:
for kage, ds in ds_dict.items():
    print(f"Saving dataset for {kage}...")
    kage_dir = analysis_dir / kage
    kage_dir.mkdir(parents=True, exist_ok=True)
    ds_file_path = kage_dir / f"{kage}.nc"
    ds.to_netcdf(ds_file_path, mode="w", engine="netcdf4", format="NETCDF4")
    print(f"Dataset for {kage} saved to {ds_file_path}.")

    img_file_path = kage_dir / f"{kage}_background.png"
    plt.imsave(img_file_path, img_dict[kage])
    print(f"Background image for {kage} saved to {img_file_path}.\n")

Saving dataset for kage10...
Dataset for kage10 saved to /ceph/neuroinformatics/neuroinformatics/sirmpilatzen/behav_data/Loukia/SmartKages/movement_analysis/kage10/kage10.nc.
Background image for kage10 saved to /ceph/neuroinformatics/neuroinformatics/sirmpilatzen/behav_data/Loukia/SmartKages/movement_analysis/kage10/kage10_background.png.

